# Exercises 3.

*Antti Härkönen*

## 1.

$$ q_j*(z_j) \propto exp(E_{-j}[log p(z_j, \bf z_{-j}, \bf x)]) $$

$$ = exp[E_{-j}(log [p(z_j \mid \bf z_{-j}, \bf x) \prod_{i \neq j}p(z_i \mid \bf x) \prod_j p(\bf x_j)])] $$

$$ = exp[E_{-j}(log [p(z_j \mid z_{-j}, x)] + \sum_{i \neq j} E_{i}log [p(z_i \mid x)] + \sum_j E_{j}log [p(\bf x_j)])] $$

$$ = exp(E_{-j}[log p(z_j \mid z_{-j}, x)]) \prod_{i \neq j} exp(E_{i}log [p(z_i \mid x)]) \prod_j exp [E_{j}log p(\bf x_j)] $$

$$ = exp(E_{-j}[log p(z_j \mid z_{-j}, x)]) \prod_{i \neq j} exp(const.) \prod_j exp (const.) $$

$$ \propto exp(E_{-j}[log p(z_j \mid \bf z_{-j}, \bf x)])$$

$$ \square $$

## 2.

$$ q(\mu, \bf \Lambda, \pi) = q(\mu \mid \Lambda, \pi) q(\Lambda \mid \pi) q(\pi) $$

$$ = q(\pi) \prod^K_{k=1} q(\mu_k \mid \Lambda_k, \pi) \prod^K_{k=1} q(\Lambda_k \mid \pi) $$

$$ = q(\pi) \prod^K_{k=1} \frac{q(\mu_k, \Lambda_k, \pi)}{q(\Lambda_k, \pi)} \times \frac{q(\Lambda_k, \pi)}{q(\pi)} $$

$$ = q(\pi) \prod^K_{k=1} \frac{q(\mu_k, \Lambda_k, \pi)}{q(\pi)} $$

There are no paths from $\pi$ to $\mu$ or $\Lambda$, so they are independent.

$$ = q(\pi) \prod^K_{k=1} \frac{q(\mu_k, \Lambda_k) p(\pi)}{q(\pi)} $$

$$ = q(\pi) \prod^K_{k=1}q(\mu_k, \Lambda_k) $$

$$ \square $$

## 3.

## 4.